In [ ]:
import pandas as pd
import time

stock_data = pd.read_csv('Final Stock Data.csv', index_col='Date')
stock_data.index = pd.to_datetime(stock_data.index)
stock_data

In [ ]:
stock_symbols = list(stock_data.columns)

ss = []
for s in stock_symbols:
    if s[:-3] == 'GATI':
        ss.append('ACLGATI')
    else:
        ss.append(s[:-3])

stock_symbols = ss

len(stock_symbols), stock_symbols

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import json

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)


# XPath templates for PE Ratios by year
xpath_templates = {
    "2023": '/html/body/div[1]/div[2]/div/div[3]/div[2]/div/div[7]/div/div/div[2]/div/table/tbody/tr[2]/td[1]',
    "2022": '/html/body/div[1]/div[2]/div/div[3]/div[2]/div/div[7]/div/div/div[2]/div/table/tbody/tr[2]/td[2]',
    "2021": '/html/body/div[1]/div[2]/div/div[3]/div[2]/div/div[7]/div/div/div[2]/div/table/tbody/tr[2]/td[3]',
    "2020": '/html/body/div[1]/div[2]/div/div[3]/div[2]/div/div[7]/div/div/div[2]/div/table/tbody/tr[2]/td[4]',
    "2019": '/html/body/div[1]/div[2]/div/div[3]/div[2]/div/div[7]/div/div/div[2]/div/table/tbody/tr[2]/td[5]',
    "2018": '/html/body/div[1]/div[2]/div/div[3]/div[2]/div/div[7]/div/div/div[2]/div/table/tbody/tr[2]/td[6]',
}

pe_ratios = {}
count = 0
for symbol in stock_symbols:
    print(symbol)
    url = f"https://www.topstockresearch.com/rt/Financial/{symbol}/PriceToEarningRatio"
    driver.get(url)
    time.sleep(2)
    
    pe_ratios[symbol] = {}
    for year, xpath in xpath_templates.items():
        time.sleep(1)
        try:
            element = driver.find_element(By.XPATH, xpath).text
            if element:
                pe_ratios[symbol][year] = element
            else:
                pe_ratios[symbol][year] = "N/A"
        except:
            pe_ratios[symbol][year] = "N/A"
    print(count)
    count += 1

driver.quit()


In [ ]:
pe_ratios

In [ ]:
pe_ratios_float = {}

for symbol, year_wise_dict in pe_ratios.items():
    pe_ratios_float[symbol] = {}
    for year, ratio in year_wise_dict.items():
        if ratio == 'N/A':
            pe_ratios_float[symbol][year] = 0.00
        else:
            pe_ratios_float[symbol][year] = float(ratio)

In [ ]:
pe_ratios_float

In [ ]:
# Save data to JSON
with open('pe_ratios.json', 'w') as file:
    json.dump(pe_ratios_float, file)